In [43]:
import pandas as pd
pd.set_option('display.max_columns', None)
X_df = pd.read_csv('./train.csv')
X_df.head()
df = pd.DataFrame(X_df)
df = df.drop(['label'], axis = 1) 


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     17176 non-null  int64  
 1   additional_fare            16974 non-null  float64
 2   duration                   16974 non-null  float64
 3   meter_waiting              16974 non-null  float64
 4   meter_waiting_fare         16974 non-null  float64
 5   meter_waiting_till_pickup  16974 non-null  float64
 6   pickup_time                17176 non-null  object 
 7   drop_time                  17176 non-null  object 
 8   pick_lat                   17176 non-null  float64
 9   pick_lon                   17176 non-null  float64
 10  drop_lat                   17176 non-null  float64
 11  drop_lon                   17176 non-null  float64
 12  fare                       17039 non-null  float64
dtypes: float64(10), int64(1), object(2)
memory usa

In [45]:
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(**kwargs):
    R = 6373.0

    lat1 = radians(kwargs['pick_lat'])
    lon1 = radians(kwargs['pick_lon'])
    lat2 = radians(kwargs['drop_lat'])
    lon2 = radians(kwargs['drop_lon'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

In [48]:
def format_df(dataFrame):
    
    df = dataFrame
    
    #calculatre the distance
    df['distance'] = [calculate_distance(**df[['pick_lat', 'pick_lon', 'drop_lat', 'drop_lon']].iloc[i].to_dict()) for i in range(df.shape[0])]
    
    #calculate the time difference between pickup and drop time in seconds
    df['time_difference'] = (pd.DatetimeIndex(df['drop_time']) - pd.DatetimeIndex(df['pickup_time'])).seconds
    
    #fill the missing values for the duration  
    df.loc[(pd.isnull(df.duration)), 'duration'] = df.time_difference

    #fill the missing values for additional_fare
    df.loc[(pd.isnull(df.additional_fare)), 'additional_fare'] = 10.5
    
    #fill the missing values for meter_waiting_till_pickup
    df.loc[(pd.isnull(df.meter_waiting_till_pickup)), 'meter_waiting_till_pickup'] = 60
    
    #fill the missing values for meter_waiting
    df.loc[(pd.isnull(df.meter_waiting)), 'meter_waiting'] = 120
    
    #fill the missing values for meter_waiting_fare
    df.loc[(pd.isnull(df.meter_waiting_fare)), 'meter_waiting_fare'] = df.meter_waiting * 1.75/60

    #fill the missing values for fare
    df.loc[(pd.isnull(df.fare)), 'fare'] = df.meter_waiting_fare + df.additional_fare + df.distance

    df = df.drop(['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1) 

    
    return df

In [62]:
format_df(df)

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,distance,time_difference
0,10.5,834.0,56.0,0.0000,64.0,270.32,5.094369,840
1,10.5,791.0,47.0,0.0000,134.0,197.85,3.169052,780
2,10.5,1087.0,80.0,0.0000,61.0,301.64,6.307375,1080
3,10.5,598.0,271.0,15.6638,68.0,82.30,0.862217,600
4,10.5,1020.0,120.0,3.5000,60.0,358.39,8.150340,1020
...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,198.26,2.106037,840
17172,10.5,2151.0,428.0,0.0000,39.0,581.23,10.871789,2160
17173,10.5,263.0,9.0,0.0000,110.0,76.20,1.045847,240
17174,10.5,858.0,115.0,0.0000,317.0,133.31,2.879981,840


In [50]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Series([], dtype: float64)

In [63]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

RANDOM_SEED = 6    # Set a random seed for reproducibility!

In [64]:
df.dtypes != "object"

tripid                        True
additional_fare               True
duration                      True
meter_waiting                 True
meter_waiting_fare            True
meter_waiting_till_pickup     True
pickup_time                  False
drop_time                    False
pick_lat                      True
pick_lon                      True
drop_lat                      True
drop_lon                      True
fare                          True
distance                      True
time_difference               True
dtype: bool

In [66]:
numeric_cols = df.columns[df.dtypes != "object"].values
print(numeric_cols)

['tripid' 'additional_fare' 'duration' 'meter_waiting'
 'meter_waiting_fare' 'meter_waiting_till_pickup' 'pick_lat' 'pick_lon'
 'drop_lat' 'drop_lon' 'fare' 'distance' 'time_difference']


,label
0,correct
1,correct
2,correct
3,correct
4,correct


C:\Users\X512\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


array(['incorrect', 'correct', 'correct', ..., 'correct', 'correct',
       'correct'], dtype=object)